In [1]:
import pandas as pd
from test import convertCatToNum, drop_rows_with_missing, normalize, encode_player_names, split_data

# on import toutes les données

df_atp = pd.read_csv('all_matches_ATP.csv')

In [2]:
df_atp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58182 entries, 0 to 58181
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Court         58182 non-null  object 
 1   Surface       58182 non-null  object 
 2   Round         58182 non-null  object 
 3   Player1_name  58182 non-null  object 
 4   Player2_name  58182 non-null  object 
 5   Player1_rank  58182 non-null  int64  
 6   Player2_rank  58182 non-null  int64  
 7   Comment       58182 non-null  object 
 8   Series        41811 non-null  float64
 9   y             58182 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 4.4+ MB


In [3]:

cat_cols = ['Court','Surface','Round',]

num_cols = ["Player1_rank","Player2_rank","Series",]


In [4]:
import numpy as np


df_processed = convertCatToNum(df=df_atp,cat_cols=cat_cols)
df = drop_rows_with_missing(df_processed,columns=["y","Series"])
df_atp = normalize(df,num_cols)


 ===     Replace categorial variable     ===
 ===   Convert categorical to numerical  ===
 ===         drop missing values         ===
 ===            Normalize data           ===


In [5]:
df_atp.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 41811 entries, 155 to 58181
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Player1_name         41811 non-null  object 
 1   Player2_name         41811 non-null  object 
 2   Player1_rank         41811 non-null  float64
 3   Player2_rank         41811 non-null  float64
 4   Comment              41811 non-null  object 
 5   Series               41811 non-null  float64
 6   y                    41811 non-null  int64  
 7   Court_Indoor         41811 non-null  uint8  
 8   Court_Outdoor        41811 non-null  uint8  
 9   Surface_Carpet       41811 non-null  uint8  
 10  Surface_Clay         41811 non-null  uint8  
 11  Surface_Grass        41811 non-null  uint8  
 12  Surface_Hard         41811 non-null  uint8  
 13  Round_1st Round      41811 non-null  uint8  
 14  Round_2nd Round      41811 non-null  uint8  
 15  Round_3rd Round      41811 non-nul

In [6]:
df_atp.head()

,Player1_name,Player2_name,Player1_rank,Player2_rank,Comment,Series,y,Court_Indoor,Court_Outdoor,Surface_Carpet,...,Surface_Grass,Surface_Hard,Round_1st Round,Round_2nd Round,Round_3rd Round,Round_4th Round,Round_Quarterfinals,Round_Round Robin,Round_Semifinals,Round_The Final
155,Pioline C.,Dosedel S.,0.005015,0.014649,Completed,1.0,1,0,1,0,...,0,1,0,1,0,0,0,0,0,0
156,Arthurs W.,Heuberger I.,0.024189,0.047609,Completed,1.0,1,0,1,0,...,0,1,0,1,0,0,0,0,0,0
157,Nestor D.,Squillari F.,0.064307,0.002848,Completed,1.0,1,0,1,0,...,0,1,0,1,0,0,0,0,0,0
158,Santoro F.,Norman M.,0.013864,0.000814,Completed,1.0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,0
159,Mamiit C.,Prinosil D.,0.036873,0.007935,Completed,1.0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,0


In [7]:
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [8]:
df_atp['Comment'].value_counts()

Completed       40248
Retired          1315
Walkover          241
Awarded             3
Disqualified        2
Sched               1
Rrtired             1
Name: Comment, dtype: int64

In [9]:
df_final = encode_player_names(df=df_atp)

=== Encoding player names ===
Encoding completed.


In [10]:
x_train, x_test, y_train, y_test = split_data(df_final)

=== Splitting data into train and test sets ===
Data split completed.


In [11]:
X_train = x_train.drop(['Comment'],axis=1)
X_test = x_test.drop(['Comment'],axis=1)

# Decision tree

In [12]:
from sklearn.tree import DecisionTreeClassifier

# Define model
model = DecisionTreeClassifier()


# define search space
param_grid = {
    'max_depth': list(range(25, 100, 10)),
    'max_features': ['auto', 'sqrt', 'log2']
}

search = RandomizedSearchCV(model, param_grid, n_iter=5, cv=10, scoring='accuracy', n_jobs=-1, random_state=1)

# execute search
result = search.fit(X_train, y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)
# {'max_features': 'auto', 'max_depth': 85}

Best Score: 0.5793020668411226
Best Hyperparameters: {'max_features': 'log2', 'max_depth': 65}


In [13]:
model = DecisionTreeClassifier(max_features='auto',max_depth=85)
# execute search
model.fit(X_train, y_train)

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\tree\_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(


DecisionTreeClassifier(max_depth=85, max_features='auto')

In [14]:
# Prediction with best model from random search 
#best_random = result.best_estimator_

y_pred_test = model.predict(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.57      0.57      0.57      6815
           1       0.58      0.57      0.57      6983

    accuracy                           0.57     13798
   macro avg       0.57      0.57      0.57     13798
weighted avg       0.57      0.57      0.57     13798



# xgboost

In [15]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Define the XGBoost model
model = xgb.XGBClassifier()

# Define the search space (parameter grid)
param_grid = {
    'n_estimators': np.arange(100, 1000, 100),  # Number of boosting rounds
    'max_depth': np.arange(3, 10),  # Maximum tree depth
    'learning_rate': [0.001,0.005,0.01, 0.1],  # Step size shrinkage used in update to prevent overfitting
    'subsample': [0.7, 0.8, 0.9, 1.0],  # Fraction of samples used for fitting the trees
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],  # Fraction of features used for fitting the trees
}

# Create a RandomizedSearchCV object
search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1, random_state=1)

# Execute the search
result = search.fit(X_train, y_train)

# Summarize the results
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [16]:
best_params =  {'subsample': 0.8, 'n_estimators': 500, 'max_depth': 4, 'learning_rate': 0.001, 'colsample_bytree': 0.8}
# Define the XGBoost model
model = xgb.XGBClassifier(**best_params)
model.fit(X_train, y_train)


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\data.py:208: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[09:30:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.001, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [17]:
# Prediction with best model from random search 
y_pred_test = model.predict(X_test)



print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.63      0.69      0.66      6815
           1       0.67      0.61      0.64      6983

    accuracy                           0.65     13798
   macro avg       0.65      0.65      0.65     13798
weighted avg       0.65      0.65      0.65     13798



# CatBoost

In [18]:

from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV

# Define the CatBoost model
model = CatBoostClassifier()

# Define the search space (parameter grid)
param_grid = {
    'iterations': [100, 200, 300],  # Number of boosting rounds
    'depth': [8,10,12],  # Maximum tree depth
    'learning_rate': [0.01, 0.1, 0.2, 0.3],  # Step size shrinkage used in update
    'subsample': [0.7, 0.8, 0.9, 1.0],  # Fraction of samples used for fitting the trees 
    'colsample_bylevel': [0.7, 0.8, 0.9, 1.0],  # Fraction of features used for fitting the trees at each level
}

# Create a RandomizedSearchCV object
search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1, random_state=1)

# Execute the search
result = search.fit(X_train, y_train)

# Summarize the results
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

In [19]:
best_params = {'subsample': 0.8, 'learning_rate': 0.01, 'iterations': 100, 'depth': 12, 'colsample_bylevel': 0.8}
model = CatBoostClassifier(**best_params)
model.fit(X_train, y_train)

0:	learn: 0.6921079	total: 417ms	remaining: 41.3s
1:	learn: 0.6908783	total: 590ms	remaining: 28.9s
2:	learn: 0.6896738	total: 699ms	remaining: 22.6s
3:	learn: 0.6885098	total: 822ms	remaining: 19.7s
4:	learn: 0.6873258	total: 985ms	remaining: 18.7s
5:	learn: 0.6861920	total: 1.15s	remaining: 18s
6:	learn: 0.6849964	total: 1.27s	remaining: 16.9s
7:	learn: 0.6839167	total: 1.37s	remaining: 15.8s
8:	learn: 0.6827981	total: 1.46s	remaining: 14.8s
9:	learn: 0.6818431	total: 1.55s	remaining: 14s
10:	learn: 0.6808176	total: 1.57s	remaining: 12.7s
11:	learn: 0.6797884	total: 1.67s	remaining: 12.2s
12:	learn: 0.6787568	total: 1.72s	remaining: 11.5s
13:	learn: 0.6777658	total: 1.91s	remaining: 11.8s
14:	learn: 0.6768361	total: 2.05s	remaining: 11.6s
15:	learn: 0.6758641	total: 2.15s	remaining: 11.3s
16:	learn: 0.6748547	total: 2.2s	remaining: 10.7s
17:	learn: 0.6739297	total: 2.3s	remaining: 10.5s
18:	learn: 0.6729620	total: 2.39s	remaining: 10.2s
19:	learn: 0.6719642	total: 2.5s	remaining: 10s

In [20]:
# Prediction with best model from random search 
# best_random = result.best_estimator_

# y_pred_test = best_random.predict(X_test)
y_pred_test = model.predict(X_test)
y_pred_test_prob = model.predict_proba(X_test)

print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.64      0.67      0.66      6815
           1       0.66      0.63      0.65      6983

    accuracy                           0.65     13798
   macro avg       0.65      0.65      0.65     13798
weighted avg       0.65      0.65      0.65     13798



# Logistic Regression

In [21]:
from sklearn.linear_model import LogisticRegression

# Define model
model = LogisticRegression()

# define evaluation
# gkf = GroupKFold(n_splits=3) if not spec as cv= in the randomized search default is 5 fold cv

# define search space
param_grid = {
    'C' : np.logspace(0, 4, num=10),
    'penalty' : ['l1', 'l2'],
    'solver' : ['liblinear', 'sag']
}

search = RandomizedSearchCV(model, param_grid, n_iter=5, cv=10, scoring='accuracy', n_jobs=-1, random_state=1)

# execute search
result = search.fit(X_train, y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

In [22]:
# # Prediction with best model from random search 
# best_random = result.best_estimator_
# best_random = model.fit(X_train, y_train)


best_params = {'solver': 'sag', 'penalty': 'l2', 'C': 1.0}
model = LogisticRegression()
model.fit(X_train,y_train)
# plot_gain(X_test=X_test,y_pred=y_pred_test,y_test=y_test, limite_cote=2)

LogisticRegression()

In [23]:
# y_pred_test = best_random.predict(X_test)
y_pred_test = model.predict(X_test)
y_pred_test_prob = model.predict_proba(X_test)

print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.50      0.75      0.60      6815
           1       0.52      0.26      0.35      6983

    accuracy                           0.50     13798
   macro avg       0.51      0.51      0.47     13798
weighted avg       0.51      0.50      0.47     13798



# RandomForestClassifier

In [24]:
# Define model
model = RandomForestClassifier()

# define evaluation
#gkf = GroupKFold(n_splits=3) if not spec as cv= in the randomized search default is 5 fold cv

# define search space
param_grid = {
    'max_depth': list(range(25, 100, 10)),
    'n_estimators': list(range(80, 200, 10))
}

search = RandomizedSearchCV(model, param_grid, n_iter=5, cv=10, scoring='accuracy', n_jobs=-1, random_state=1)

# execute search
result = search.fit(X_train, y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

In [25]:
best_params = {'n_estimators': 120, 'max_depth': 55}
model = RandomForestClassifier(**best_params)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=55, n_estimators=120)

In [26]:
# Prediction with best model from random search 
# best_random = result.best_estimator_
# y_pred_test = best_random.predict(X_test)

y_pred_test = model.predict(X_test)
y_pred_test_prob = model.predict_proba(X_test)
print(classification_report(y_test, y_pred_test))


              precision    recall  f1-score   support

           0       0.62      0.65      0.64      6815
           1       0.65      0.62      0.63      6983

    accuracy                           0.64     13798
   macro avg       0.64      0.64      0.64     13798
weighted avg       0.64      0.64      0.64     13798



# MLPClassifier

In [27]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [28]:
mlp = MLPClassifier()

# Il s'agit d'une sortie binaire 
# donc la fonction d’activation des neurones de la couche de sortie est 'logistic'
mlp.out_activation_ = 'logistic'
mlp.n_outputs_ = 1

In [29]:
# Grille de recherche d'hyperparamètres
param_grid = {
    'hidden_layer_sizes': [(30, 20, 15, 25),(50, 40, 30,20)],
    'activation':['relu', 'tanh', 'logistic'],#['relu'],
    'solver':  ['adam', 'lbfgs', 'sgd'],# ['adam'],
}

In [30]:
grid_search = GridSearchCV(mlp, param_grid, n_jobs=-1, cv=3)
grid_search.fit(X_train, y_train)

In [ ]:
# best_params = {'activation': 'relu', 'hidden_layer_sizes': (30, 20, 15, 25), 'solver': 'lbfgs'}

# Obtenir la meilleure configuration d'hyperparamètres
best_params = grid_search.best_params_
# meilleur parametre
print(f'Best parameters found : {best_params}')

Best parameters found : {'activation': 'tanh', 'hidden_layer_sizes': (30, 20, 15, 25), 'solver': 'lbfgs'}


In [ ]:
best_mlp = MLPClassifier(**best_params)
best_mlp.fit(X_train, y_train)

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(activation='tanh', hidden_layer_sizes=(30, 20, 15, 25),
              solver='lbfgs')

In [ ]:
y_pred_test = best_mlp.predict(X_test)
y_pred_test_prob = best_mlp.predict_proba(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.69      0.66      0.67      6542
           1       0.69      0.73      0.71      7006

    accuracy                           0.69     13548
   macro avg       0.69      0.69      0.69     13548
weighted avg       0.69      0.69      0.69     13548

